In [28]:
import pandas as pd
import numpy as np
from pathlib import Path
import math
from preprocess import build_course_id2attr, build_id_lookup

In [29]:
output_dir = Path('output')
input_dir = Path('hahow/data')
cache_dir = Path('cache')

In [30]:
train_dataset = pd.read_csv(input_dir / 'train.csv')
val_dataset = pd.read_csv(input_dir / 'val_seen.csv')
courses = pd.read_csv(input_dir / 'courses.csv')
users = pd.read_csv(input_dir / 'users.csv')

In [41]:
len(train_dataset)

59737

In [31]:
lookup = build_id_lookup(train_dataset)

user #: 59737
course #: 664


In [37]:
related_courses = {}

def update_related_course(idx1: int, idx2: int):
    if idx1 in related_courses:
        related_courses[idx1].add(idx2)
    else:
        related_courses[idx1] = set([idx2])

for i, row in train_dataset.iterrows():
    course_idxs = [lookup['course_id2idx'][course_id] for course_id in row.course_id.split()]
    for j, idx1 in enumerate(course_idxs):
        for k, idx2 in enumerate(course_idxs):
            if j < k:
                update_related_course(idx1, idx2)
                update_related_course(idx2, idx1)

len(related_courses)

662

In [23]:
users

,user_id,gender,occupation_titles,interests,recreation_names
0,54ccaa73a784960a00948687,female,NaN,"職場技能_創業,藝術_電腦繪圖,設計_介面設計,設計_動態設計,設計_平面設計,投資理財_投...",NaN
1,54dca4456d7d350900e86bae,male,NaN,"設計_動態設計,設計_平面設計,設計_應用設計,程式_程式入門,程式_程式語言,藝術_角色設...",NaN
2,54e421bac5c9c00900cd8d47,female,NaN,"設計_平面設計,職場技能_資料彙整,藝術_繪畫與插畫,行銷_數位行銷,職場技能_文書處理,職...",NaN
3,54e961d4c5c9c00900cd8d84,other,金融業,"投資理財_理財,攝影_影像創作,投資理財_投資觀念,藝術_更多藝術,音樂_樂器,投資理財_金...",NaN
4,54e9b744c5c9c00900cd8d8a,other,"資訊科技,法律、社會及文化專業,非營利組織","程式_網頁前端,投資理財_理財,投資理財_投資觀念,程式_程式語言,設計_設計理論,投資理財...","政治經濟,社會服務,舞台劇,電影"
...,...,...,...,...,...
130561,62e09de8fc3d3500060d4211,female,NaN,"語言_英文,設計_介面設計,設計_網頁設計,設計_設計理論,程式_軟體程式開發與維護,行銷_...",NaN
130562,62f0823a8c4414000667c592,NaN,NaN,NaN,NaN
130563,631b86242145060007efc7dd,NaN,NaN,NaN,NaN
130564,6331648104ed0f000610dfd2,male,公務人員,"投資理財_理財,攝影_影像創作,攝影_後製剪輯,攝影_商業攝影,投資理財_投資觀念","旅行旅遊,運動健身,金融理財,電影"


In [24]:
def build_attr_id2idx(df: pd.DataFrame, column_name: str):
    ids = set()
    for ids_str in users[column_name]:
        for id in str(ids_str).split(','):
            ids.add(id)
    id2idx = {}
    for i, id in enumerate(sorted(ids)):
        id2idx[id] = i
    return id2idx

columns = {
    'gender_id2idx': 'gender',
    'occupation_id2idx': 'occupation_titles',
    'interest_id2idx': 'interests',
    'recreation_id2idx': 'recreation_names',
}

user_lookup = {}

for lookup_type, column in columns.items():
    user_lookup[lookup_type] = build_attr_id2idx(users, column)

user_lookup



{'gender_id2idx': {'female': 0, 'male': 1, 'nan': 2, 'other': 3},
 'occupation_id2idx': {'nan': 0,
  '公務人員': 1,
  '其他': 2,
  '出版業': 3,
  '家管': 4,
  '廣告傳播': 5,
  '教學專業': 6,
  '服務業': 7,
  '法律、社會及文化專業': 8,
  '營建工程': 9,
  '科技業': 10,
  '職業軍人': 11,
  '自由業': 12,
  '藝文設計': 13,
  '製造業': 14,
  '資訊科技': 15,
  '農林漁牧': 16,
  '退休': 17,
  '醫療': 18,
  '金融業': 19,
  '非營利組織': 20},
 'interest_id2idx': {'nan': 0,
  '人文_文學': 1,
  '人文_更多人文': 2,
  '人文_社會科學': 3,
  '手作_刺繡': 4,
  '手作_手作小物': 5,
  '手作_手工印刷': 6,
  '手作_手工書': 7,
  '手作_更多手作': 8,
  '手作_模型': 9,
  '手作_氣球': 10,
  '手作_篆刻': 11,
  '投資理財_投資觀念': 12,
  '投資理財_更多投資理財': 13,
  '投資理財_比特幣': 14,
  '投資理財_理財': 15,
  '投資理財_量化交易': 16,
  '投資理財_金融商品': 17,
  '攝影_動態攝影': 18,
  '攝影_商業攝影': 19,
  '攝影_影像創作': 20,
  '攝影_影視創作': 21,
  '攝影_後製剪輯': 22,
  '攝影_攝影理論': 23,
  '攝影_更多攝影': 24,
  '生活品味_壓力舒緩': 25,
  '生活品味_寵物': 26,
  '生活品味_居家': 27,
  '生活品味_心靈成長與教育': 28,
  '生活品味_數學': 29,
  '生活品味_更多生活品味': 30,
  '生活品味_烹飪料理與甜點': 31,
  '生活品味_親子教育': 32,
  '生活品味_護膚保養與化妝': 33,
  '生活品味_運動': 34,
  '生活品味_靈性發展'

In [25]:
def build_n_hot_encode_str(id2idx: dict, ids_str):
    n = len(id2idx)
    xs = [0] * n
    for id in str(ids_str).split(','):
        xs[id2idx[id]] = 1
    return xs

def build_user_encode():
    user_id2encode = {}
    for i, row in users.iterrows():
        encoded_list = []
        for lookup_type, column in columns.items():
            n_hot = build_n_hot_encode_str(user_lookup[lookup_type], row[column])
            encoded_list += n_hot
        user_id2encode[row.user_id] = encoded_list
    return user_id2encode

In [26]:
user_id2encode = build_user_encode()

153

In [ ]:
len(user_id2encode['54ccaa73a784960a00948687'])